# Load data

In [2]:
# pip install mlflow

In [5]:
import os
import pandas as pd
import numpy as np

# data_path = '../Data_preprocessing/'
training_data_path = 'research_training_set_with_concatenated_reviews_business_categories_and_feature_vectors.csv'
training_data = pd.read_csv(training_data_path)
# training_data.head()

# data_path = '../Data_preprocessing/'
validation_data_path = 'research_validation_set_with_concatenated_reviews_business_categories_and_feature_vectors.csv'
validation_data = pd.read_csv(validation_data_path)
# validation_data.head()

In [6]:
# Training and Validation Data preprocessing

columns_to_train = ['user_feature_vector', 'business_feature_vector']
# columns_to_train = ['user_feature_vector', 'business_feature_vector', "business_categories_feature_vector"]

concatenated_vectors = []
for i in range(len(training_data)):
    col_vectors = []
    for col in columns_to_train:
        col_vectors.append(np.array(eval(training_data[col][i])))
    concatenated_vector = np.concatenate(col_vectors)
    concatenated_vectors.append(concatenated_vector)

X_train = np.array(concatenated_vectors)
y_train = np.array(training_data['stars'])

concatenated_vectors = []
for i in range(len(validation_data)):
    col_vectors = []
    for col in columns_to_train:
        col_vectors.append(np.array(eval(validation_data[col][i])))
    concatenated_vector = np.concatenate(col_vectors)
    concatenated_vectors.append(concatenated_vector)

X_valid = np.array(concatenated_vectors)
y_valid = np.array(validation_data['stars'])

# Load written code

In [58]:
from pathlib import Path
import nbformat

def load_notebook(notebook_path):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)
    code_cells = [cell.source for cell in nb.cells if cell.cell_type == 'code']
    exec('\n'.join(code_cells), globals())

# import written function and variable

parent_directory = Path('../../../')
# parent_directory = Path('../../')
model_zoo_path = parent_directory / 'model_zoo.ipynb'
model_training_validation_testing_function_path = parent_directory / 'model_train_val_test_function.ipynb'
# model_training_validation_params_path = parent_directory / 'model_train_val_params.ipynb'

load_notebook(model_zoo_path)
load_notebook(model_training_validation_testing_function_path)
# load_notebook(model_training_validation_params_path)

# FM experiments

In [59]:
input_dim = len(X_train[0])
embedding_size = 768
field_num = len(columns_to_train)

FM_params = {
    'hyperparameters': {
        'input_dim': input_dim, 
#         'factors_num': [5, 10, 15],
#         'factors_num': [i+1 for i in range(20)]
#         'factors_num': [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#         'factors_num': [2**i for i in range(3, 7)],
        'factors_num': 8
    },
    
    'task_type': 'Regression', 'loss_type': 'MSE', 'optimizer_type': 'Adam', 
#   'dl_learning_rate': [0.01, 0.02, 0.001], 
    'dl_learning_rate': 0.001,
    'epochs_num': 100, 
#   'batch_size': [100, 500, 1000]
    'batch_size': 512
    
}

In [55]:
import mlflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")
mlflow.set_experiment("origin_text_f4/FM")

mlflow_model_training_validtion_or_testing(X_train, y_train, X_valid, y_valid, 
                             "FM", **FM_params, save_records=True, testing=False)

2024/04/08 16:53:55 INFO mlflow.tracking.fluent: Experiment with name 'origin_text_f4/FM' does not exist. Creating a new experiment.


Start training FM model ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start 

(              Timestamp Model_name    Training_for  FM_input_dim  \
 0   2024-04-08 16:53:56         FM  for_validation          1536   
 1   2024-04-08 16:53:56         FM  for_validation          1536   
 2   2024-04-08 16:53:56         FM  for_validation          1536   
 3   2024-04-08 16:53:56         FM  for_validation          1536   
 4   2024-04-08 16:53:56         FM  for_validation          1536   
 ..                  ...        ...             ...           ...   
 95  2024-04-08 16:54:09         FM  for_validation          1536   
 96  2024-04-08 16:54:09         FM  for_validation          1536   
 97  2024-04-08 16:54:09         FM  for_validation          1536   
 98  2024-04-08 16:54:09         FM  for_validation          1536   
 99  2024-04-08 16:54:10         FM  for_validation          1536   
 
     FM_factors_num   Task_type Loss_type Optimizer_type  DL_learning_rate  \
 0                8  Regression      RMSE           Adam             0.001   
 1            

# AutoInt experiments

In [56]:
# AutoInt
# field_dims, embed_dim, atten_embed_dim, num_heads, num_layers, mlp_dims, dropouts, has_residual=True
# embed_dim=16, atten_embed_dim=64, num_heads=2, num_layers=3, mlp_dims=(400, 400), dropouts=(0, 0, 0)

# Paper:
# embed_dim is set to 16, batch size set to 1024
# num_head is 2, num_layer(interaction layer) is 3, num of hidden units (interaction layer) is 32
# test dropout from 0.1 ~ 0.9
# optimizer is Adam
# test num_layer(interaction layer) from 0 ~ 4, 1 increase dramaticaly, 1 ~ 4 become stable
# test atten_embed_dim 8, 16, 24, 32, movie len is getting better, for KDD12, 24 is best, then decrease
# hidden units shape is set to (1, 200) or (4, 100)
# residaul is crucial

input_dim = len(X_train[0])
embedding_size = 768
field_num = len(columns_to_train)

AutoInt_params = {
    'hyperparameters': {
        'field_dims': [embedding_size for i in range(field_num)], 
#         'embed_dim': [768], 
        'embed_dim': 768, 
#         'atten_embed_dim': [(64, 32)],
#         'atten_embed_dim': [2**i for i in range(2, 7)],
        'atten_embed_dim': 32,
#         "num_heads": [2],
        "num_heads": 2,
#         "num_layers": [3],
#         "num_layers": [i+1 for i in range(5)],
        "num_layers": 3,
#         'mlp_dims': [(16, 16), (400, 400)],
#         'mlp_dims': [tuple([size] * num_layers) for num_layers in range(1, 6) for size in [i*100 for i in range(1, 6)]],
        'mlp_dims': (256, 256, 256),
#         'dropouts': [(0.5, 0.5, 0.5)],
#         'dropouts': [tuple([size * 0.1] * 3) for size in range(1, 10)],
        'dropouts': (0.0, 0.0, 0.0),
        "has_residual": True
    },
    
    'task_type': 'Regression', 'loss_type': 'MSE', 'optimizer_type': 'Adam', 
#         'dl_learning_rate': [0.01, 0.02, 0.001], 
#     'dl_learning_rate': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
    'dl_learning_rate': 0.001,
#     'epochs_num': [10, 20, 30], 
    'epochs_num': 100,
#         'batch_size': [100, 500, 1000]
#     'batch_size': [128, 256, 512, 1024]
    'batch_size': 512
    
}

In [57]:
import mlflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")
mlflow.set_experiment("origin_text_f4/AutoInt")

mlflow_model_training_validtion_or_testing(X_train, y_train, X_valid, y_valid, 
                             "AutoInt", **AutoInt_params, save_records=True, testing=False)

2024/04/08 16:54:10 INFO mlflow.tracking.fluent: Experiment with name 'origin_text_f4/AutoInt' does not exist. Creating a new experiment.


Start training AutoInt model ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
Start validating ...
S

(              Timestamp Model_name    Training_for AutoInt_field_dims  \
 0   2024-04-08 16:54:10    AutoInt  for_validation         [768, 768]   
 1   2024-04-08 16:54:10    AutoInt  for_validation         [768, 768]   
 2   2024-04-08 16:54:11    AutoInt  for_validation         [768, 768]   
 3   2024-04-08 16:54:11    AutoInt  for_validation         [768, 768]   
 4   2024-04-08 16:54:11    AutoInt  for_validation         [768, 768]   
 ..                  ...        ...             ...                ...   
 95  2024-04-08 16:54:44    AutoInt  for_validation         [768, 768]   
 96  2024-04-08 16:54:45    AutoInt  for_validation         [768, 768]   
 97  2024-04-08 16:54:45    AutoInt  for_validation         [768, 768]   
 98  2024-04-08 16:54:45    AutoInt  for_validation         [768, 768]   
 99  2024-04-08 16:54:46    AutoInt  for_validation         [768, 768]   
 
     AutoInt_embed_dim  AutoInt_atten_embed_dim  AutoInt_num_heads  \
 0                 768                  